In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Tensorflow and other requirements**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D , Conv2D , Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import categorical_crossentropy , Accuracy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical

# ****Loading Dataset****

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

# **Defining Trainiing Labels and Testing Labels**

In [ ]:
xtrain = train.drop(['label'],axis = 1).astype('int32')
ytrain = train['label'].astype('float32')
xtest = test.astype('float32')

In [ ]:
xtrain.shape

In [ ]:
ytrain.shape

# **Reshaping and Normalizing**

In [ ]:
xtrain = xtrain.values.reshape(-1, 28, 28, 1)
xtest = xtest.values.reshape(-1, 28, 28, 1)
xtest = xtest/255.
xtrain = xtrain/255.

In [ ]:
ytrain = to_categorical(ytrain,10)
ytrain.shape

# **Code For Model**

In [ ]:
model = Sequential([
    Conv2D(32 , (3,3) , activation = 'relu' , input_shape = (28,28,1)), #first layer as convolution , 3x3 kernal , activation function is relu , input shape of image is 28,28,1
    Conv2D(32 , (3,3) , activation = 'relu'),
    MaxPool2D(),
    
    
    Conv2D(64 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(64 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Conv2D(128 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(128 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Conv2D(256 , (3,3) , activation = 'relu',padding='Same' ),
    Conv2D(256 , (3,3) , activation = 'relu',padding='Same' ),
    MaxPool2D(),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(xtrain,ytrain, batch_size=128, epochs=10)

# **Predictions**

In [ ]:
preds = model.predict(xtest)
preds = np.argmax(preds,axis=1)
preds = pd.Series(preds,name='Label')

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = 'ImageId'),preds],axis = 1)
submission.to_csv("submission.csv",index = False)

In [ ]:
!kaggle competitions submit -c Digit-Recognizer -f submission.csv -m "Message"